In [64]:
import pandas as pd
from collections import defaultdict
import json

In [9]:
xls = pd.ExcelFile('../resources/WorldEnergyBalancesHighlights2021.xlsx')

df = pd.read_excel(xls, 'TimeSeries_1971-2020', header=1)
df

,Country,Product,Flow,NoCountry,NoProduct,NoFlow,1971,1972,1973,1974,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020 Provisional
0,Australia,"Coal, peat and oil shale",Production (PJ),01. Australia,"01. Coal, peat and oil shale",01. Production (PJ),1368.251838,1648.414248,1685.223676,1640.362098,...,9782.067241,10272.36221,11074.77871,11950.78794,12501.00154,12223.55369,12264.16248,12034.6322,12596.41582,12331.64135
1,Australia,"Coal, peat and oil shale",Imports (PJ),01. Australia,"01. Coal, peat and oil shale",02. Imports (PJ),0,0,0,0,...,6.755636,3.635428,2.57749,5.287409,6.978797,6.829994,7.916741,12.735731,17.037944,19.629498
2,Australia,"Coal, peat and oil shale",Exports (PJ),01. Australia,"01. Coal, peat and oil shale",03. Exports (PJ),-541.424001,-614.718001,-738.939398,-745.382946,...,-7720.391726,-8171.019358,-9101.059456,-10166.8869,-10634.4478,-10552.5051,-10248.0298,-10333.3662,-10629.0366,-10541.275
3,Australia,"Coal, peat and oil shale",Total energy supply (PJ),01. Australia,"01. Coal, peat and oil shale",04. Total energy supply (PJ),884.29747,923.250159,945.54188,928.32307,...,2013.814178,1968.848884,1831.204342,1736.61351,1797.035545,1863.051458,1872.030679,1797.650326,1744.105366,1648.774748
4,Australia,"Coal, peat and oil shale","Electricity, CHP and heat plants (PJ)",01. Australia,"01. Coal, peat and oil shale","05. Electricity, CHP and heat plants (PJ)",-516.538554,-569.92192,-577.900491,-617.948468,...,-1789.197177,-1769.932048,-1650.413411,-1561.052148,-1622.70676,-1668.865761,-1651.517169,-1573.934618,-1531.523087,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6155,IEA and Accession/Association countries,Total,Electricity output (GWh),54. IEA and Accession/Association countries,11. Total,13. Electricity output (GWh),..,..,..,..,...,17887289.48,18253458.88,18833000.52,19281187.98,19434697.21,19950693.71,20589646.76,21366327.3,21641349.85,21619592.25
6156,World,Fossil fuels,Electricity output (GWh),55. World,09. Fossil fuels,13. Electricity output (GWh),3905413.008,4241816.367,4606277.139,4552612.92,...,15126682.35,15451913.86,15781936.88,16046172.63,16094265.01,16339136.28,16687929.51,17081705.24,17007627.45,..
6157,World,Nuclear,Electricity output (GWh),55. World,05. Nuclear,13. Electricity output (GWh),111087,152475,203196,272711,...,2582639.326,2460285.54,2479105.211,2535404.549,2570070.656,2608401.808,2636117.238,2709100.699,2789694.402,2678855.226
6158,World,Renewable sources,Electricity output (GWh),55. World,10. Renewable sources,13. Electricity output (GWh),1240596.11,1304208.694,1321597.846,1460175.28,...,4410656.504,4729751.103,5060522.017,5302668.714,5511268.895,5891550.903,6250190.754,6679673.151,7014495.554,..


In [70]:
data = {}
years = range(1971, 2020)
print(list(years))
# year 2020 is provisional and lacks so much data that it's better to skip

def fmt(s):
    return s.lower().replace(' ', '_').replace(',', '').replace('(', '').replace(')', '')

df.Country = df.Country.apply(fmt)
df.Product = df.Product.apply(fmt)
df.Flow = df.Flow.apply(fmt)

countries = df['Country'].unique()
for country in countries:
    df_country = df.loc[df['Country'] == country]
    for index, row in df_country.iterrows():
        product = row.Product
        flow = row.Flow
        if country not in data:
            data[country] = {}

        if product not in data[country]:
            data[country][product] = {}

        if flow not in data[country][product]:
            data[country][product][flow] = {}

        value = row[year]
        if value == '..':
            value = 0

        for year in years:
            data[country][product][flow][year] = value

data

with open('../resources/iea_country_energy_mix.json', 'w') as f:
    json.dump(data, f, indent=2)


[1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


In [82]:
data = {}
years = range(1971, 2020)
print(list(years))
# year 2020 is provisional and lacks so much data that it's better to skip

def fmt(s):
    return s.lower().replace(' ', '_').replace(',', '').replace('(', '').replace(')', '')

df.Country = df.Country.apply(fmt)
df.Product = df.Product.apply(fmt)
df.Flow = df.Flow.apply(fmt)

countries = df['Country'].unique()
for country in countries:
    df_country = df.loc[df['Country'] == country]
    for index, row in df_country.iterrows():

        product = row.Product
        flow = row.Flow
        if country not in data:
            data[country] = {}

        for year in years:
            if year not in data[country]:
                data[country][year] = {}
            
            yearvalue = row[year]
            if yearvalue == '..':
                yearvalue = 0

            if product not in data[country][year]:
                data[country][year][product] = {}
            
            data[country][year][product][flow] = yearvalue


        # if product not in data[country]:
        #     data[country][product] = {}

        # if flow not in data[country][product]:
        #     data[country][product][flow] = {}

        # value = row[year]
        # if value == '..':
        #     value = 0

        # for year in years:
        #     data[country][product][flow][year] = value



d = {
    "countries": list(countries),
    "years": list(map(str, years)),
    "data": data,
}


with open('../resources/iea_country_energy_mix.json', 'w') as f:
    json.dump(d, f, indent=2)

[1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


In [84]:
data = {}
years = range(1971, 2020)
print(list(years))
# year 2020 is provisional and lacks so much data that it's better to skip

def fmt(s):
    return s.lower().replace(' ', '_').replace(',', '').replace('(', '').replace(')', '')

df.Country = df.Country.apply(fmt)
df.Product = df.Product.apply(fmt)
df.Flow = df.Flow.apply(fmt)

countries = df['Country'].unique()
for country in countries:
    df_country = df.loc[df['Country'] == country]
    for index, row in df_country.iterrows():

        product = row.Product
        flow = row.Flow

        if flow != "total_final_consumption_pj":
            continue

        if country not in data:
            data[country] = {}

        for year in years:
            if year not in data[country]:
                data[country][year] = {}
            
            yearvalue = row[year]
            if yearvalue == '..':
                yearvalue = 0
            
            data[country][year][product] = yearvalue

d = {
    "countries": list(countries),
    "years": list(map(str, years)),
    "data": data,
}


with open('../resources/iea_country_energy_mix.json', 'w') as f:
    json.dump(d, f, indent=2)

[1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
